# Kobert 사용 baseline

In [1]:
!pip install konlpy
!pip install transformers==3
!pip install gluonnlp pandas tqdm
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install wandb
!pip install pytorchtools

  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached tokenizers-0.8.0rc4-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.0 which is incompatible.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t280fbke
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t280fbke
  Using cached transformers-4

In [2]:
import pandas as pd
import numpy as np
import json
import ast
from konlpy.tag import Okt, Hannanum, Komoran, Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import wandb
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import random
import copy
import time
import gc
import re
import torch

import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from torch.optim import Adam

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import gc

import matplotlib.pyplot as plt

In [3]:
PATH="/content/drive/MyDrive/학교 프로젝트/2022 슈퍼챌린지 SW 해커톤"

In [4]:
def make_category(x):
  if int(x[1])<=3:
    return int(x[1])-1
  elif int(x[1])==4:
    return 1
  else:
    return int(x[1])-2

with open(f"{PATH}/data/train/감성대화말뭉치(최종데이터)_Training.json",'r') as datafile:
    data=json.load(datafile)
train_data=pd.json_normalize(data)
train_data=pd.DataFrame(train_data)
data=""
with open(f"{PATH}/data/test/감성대화말뭉치(최종데이터)_Validation.json",'r') as datafile:
    data=json.load(datafile)
test_data=pd.json_normalize(data)
test_data=pd.DataFrame(test_data)

train_data["context"]=train_data['talk.content.HS01']+" "+train_data['talk.content.HS02']+" "+train_data['talk.content.HS03']
test_data["context"]=test_data['talk.content.HS01']+" "+test_data['talk.content.HS02']+" "+test_data['talk.content.HS03']
used_col=['profile.persona-id','profile.emotion.type', 'context']
for col in train_data.columns:
  if col not in used_col:
    train_data.drop(columns=col,inplace=True)
    test_data.drop(columns=col,inplace=True)

train_data.columns=["id","emotion","context"]
test_data.columns=["id","emotion","context"]

train_data["context"]=train_data["context"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['context'].replace('', np.nan, inplace=True)
test_data["context"]=test_data["context"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['context'].replace('', np.nan, inplace=True)
  
train_data["emotion"]=train_data["emotion"].apply(make_category)
test_data["emotion"]=test_data["emotion"].apply(make_category)

In [5]:
test_data["emotion"].unique()

array([0, 1, 2, 3, 4])

In [6]:
train=train_data.values.tolist()
test=test_data.values.tolist()

In [7]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
len(test)

5122

In [9]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# Setting parameters
max_len = 200 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 12
warmup_ratio = 0.1
num_epochs = 100
max_grad_norm = 1
log_interval = 2000
learning_rate = 5e-5
threshold=3

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


[model 참고 URL](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)

In [10]:
class emotionDataset(Dataset):
    ''' context&emotion dataset '''
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

emotion_train_dataset = emotionDataset(train,2, 1, tok, max_len, True, False)
emotion_valid_dataset = emotionDataset(test,2, 1, tok, max_len, True, False)

train_dataloader = DataLoader(emotion_train_dataset, batch_size=batch_size, num_workers=2)
valid_dataloader = DataLoader(emotion_valid_dataset, batch_size=batch_size, num_workers=2)

In [11]:
print(len(emotion_train_dataset),len(train_dataloader))

40827 3403


In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [13]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [14]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [15]:
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=3, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            print('terminating because of early stopping!')
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [ ]:
# 모델 학습 시작
previous_loss=-1
previous_acc=0
accm_down=0
wandb.init(project="2022SCHackathon", entity="yerim")
wandb.watch(model)
early_stopping = EarlyStopping(patience=3)
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train_acc=train_acc / (batch_id+1)
    train_loss=loss.data.cpu().numpy()
    with torch.no_grad():
      model.eval() # 평가 모드로 변경
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
      print("epoch {} testloss {} test acc {}".format(e+1,loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    wandb.log({"epoch":e,"accuracy": train_acc,"val_accuracy":test_acc/(batch_id+1),"val_loss":loss.data.cpu().numpy(),"loss":train_loss})
    if early_stopping.step(loss.data.cpu().numpy()):
        break  # early stop criterion is met, we can stop now
 #     torch.save(model,f'{PATH}/models/best_model_{e+1}epoch_maxlen_200.pt')

wandb: Currently logged in as: yerim (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/3403 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7488702535629272 train acc 0.08333333333333333
epoch 1 batch id 2001 loss 1.3221477270126343 train acc 0.33608195902048976
